In [6]:
import numpy as np  # Sayısal hesaplamalar için
import pandas as pd  # Veri çerçeveleriyle çalışmak için
import matplotlib.pyplot as plt  # Grafikler çizmek için
from sklearn.linear_model import LinearRegression, LogisticRegression  # Doğrusal regresyon modeli için
from sklearn.model_selection import train_test_split  # Veriyi eğitim ve test kümelerine bölmek için
from sklearn.metrics import r2_score, mean_squared_error  # Performans metrikleri için
import seaborn as sns  # Görselleştirmeler için
%matplotlib inline
df = pd.read_csv('datasets/survey_results_public.csv')
schema = pd.read_csv('datasets/survey_results_schema.csv')
def create_dummy_df(df, cat_cols, dummy_na):
    for col in  cat_cols:
            # Her bir kategorik sütun için kukla değişken ekleyin, orijinal sütunu çıkarın
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df
def clean_fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.3, rand_state=42):
    df  = df.dropna(subset=[response_col], axis=0)
    df = df.dropna(how='all', axis=1)
    df = create_dummy_df(df, cat_cols, dummy_na)
    fill_mean = lambda col: col.fillna(col.mean())
    df = df.apply(fill_mean, axis=0)
    X = df.drop(response_col, axis=1)
    y = df[response_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)
    lm_model = LinearRegression() # Instantiate
    lm_model.fit(X_train, y_train) #Fit
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test


scores = []
Output_name = 'JobSatisfaction'
for column in df.columns:
    try:
        work_env = df[[column, Output_name]]
        cat_df = work_env.select_dtypes(include=['object']).copy()
        test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(work_env, Output_name, cat_df, dummy_na=True)
        scores.append({column, train_score})
        #print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))
    except:
        scores.append(0)
print(scores)

In [7]:
print(scores)

[{0.002386198109234239, 'ProgramHobby'}]
